In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
#mpl.rc('text', usetex=True)
from matplotlib import pyplot as plt
import pyGadget
sns.set_context('poster')

In [ ]:
sim = pyGadget.sim.Simulation('stampede/vanilla',length='AU')

In [ ]:
snap = sim.load_snapshot(355)

In [ ]:
n = snap.gas.get_number_density()
T = snap.gas.get_temperature()
Lj = snap.gas.get_jeans_length()
snap.close()

In [ ]:
n,T,Lj = pyGadget.analyze.data_slice(n >= .95, n,T,Lj)

In [ ]:
def N_p(n):
    exponent = 0.532301 * np.log10(n) + 19.636552
    return 10**exponent

def N_eq(n):
    exponent = 0.626204 * np.log10(n) + 19.573490
    return 10**exponent

Omega_pole = 1.840282
fourpi = 4 * np.pi
a = 2 * Omega_pole / fourpi
b = (fourpi - 2*Omega_pole)/fourpi
def N_eff(n):
    x = N_p(n)
    y = N_eq(n)
    return a*x + b*y

Neff = N_eff(n)
Np = N_p(n)

In [ ]:
plt.loglog(n,n*Lj, 'o', ms=2, label=r'$n \times L_J$')
plt.plot(n, Neff, label=r'$N_{\rm eff}$')
plt.plot(n, Np, label=r'$N_{\rm pole}$')
plt.legend(loc=0)
plt.xlim(1e0,1e12)
plt.ylabel('Column Density [cm$^{-2}$]')
plt.xlabel('Number Density [cm$^{-3}$]')
plt.savefig("figures/optical_depth/column_density.png", bbox_inches="tight")

In [ ]:
plt.loglog(n,Lj, 'o', ms=2, label=r'$L_J$')
plt.plot(n, Neff/n, label=r'$N_{\rm eff} / n$')
plt.plot(n, Np/n, label=r'$N_{\rm pole} / n$')
plt.legend(loc=0)
plt.xlim(1e0,1e12)
plt.ylabel('Estimated Cloud Depth [cm]')
plt.xlabel('Number Density [cm$^{-3}$]')

In [ ]:
plt.semilogx(n, Neff/(n*Lj), 'o', ms=2)
plt.axhline(1)

In [ ]:
#First, define beta.
def beta(epsilon):
    mHc2 = 938272046 #rest energy of a proton in eV
    return np.sqrt(1 - (epsilon/mHc2 + 1)**(-2))

#Define f(epsilon).
def f(epsilon):
    B0 = 0.01
    B = beta(epsilon)
    return (1 + 0.0185 * np.log(B)) * 2*B**2 / (B0**3 + 2*B**3)

def dedt(n, epsilon):
    return 1.82e-7 * n * f(epsilon)

def Dp(n,epsilon):
    return beta(epsilon) * 2.99e10 * epsilon / dedt(n,epsilon)

In [ ]:
energy = np.logspace(6,12, 100)
for nd in np.logspace(0,12, 6):
    plt.loglog(energy, Dp(nd,energy)/3.08e18)

In [ ]:
plt.loglog(n,Lj, 'o', ms=2, label=r'$L_J$')
plt.plot(n, Neff/n, label=r'$N_{\rm eff} / n$')
plt.plot(n, Np/n, label=r'$N_{\rm pole} / n$')
plt.plot(n, Dp(n,1e6), label=r'D$_{\rm p}(\epsilon = 10^6 {\rm eV})$')
plt.plot(n, Dp(n,1e8), label=r'D$_{\rm p}(\epsilon = 10^8 {\rm eV})$')
plt.legend(loc=0)
plt.xlim(1e0,1e12)
plt.ylabel('Cloud Depth [cm]')
plt.xlabel('Number Density [cm$^{-3}$]')
plt.savefig("figures/optical_depth/cloud_depth.png", bbox_inches="tight")

In [ ]:
plt.loglog(n, Lj/Dp(n,1e6), 'o', ms=2, label=r'L$_J$/D$_{\rm p}(\epsilon = 10^6 {\rm eV})$')
plt.loglog(n, Neff/n/Dp(n,1e6), label=r'($N_{\rm eff} / n$)/D$_{\rm p}(\epsilon = 10^6 {\rm eV})$')
plt.loglog(n, Lj/Dp(n,1e8), 'o', ms=2, label=r'L$_J$/D$_{\rm p}(\epsilon = 10^8 {\rm eV})$')
plt.loglog(n, Neff/n/Dp(n,1e8), label=r'($N_{\rm eff} / n$)/D$_{\rm p}(\epsilon = 10^8 {\rm eV})$')
plt.axhline(1, linestyle='--', color='k')
plt.legend(loc=0)
plt.xlim(1e0,1e12)
plt.ylabel('Optical Depth')
plt.xlabel('Number Density [cm$^{-3}$]')
plt.savefig("figures/optical_depth/optical_depth.png", bbox_inches="tight")